In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from VGGs import *
from MobileNetV3 import *
from engine import *
from efficientNet import *

change_to_disk()
data_dir = Path("clean_split_1k/")
models_path = Path('Models/')
saves = Path('Trainning_results_saves/')

In [2]:
data_transform = transforms.Compose([
    # Resize our images to 224x224
    transforms.Resize(size=(224, 224)),
    
    # Rotate the images randomly (just to make it as independent from the position of the pipeline)
    transforms.RandomRotation(180),

    # Random crop
    transforms.RandomCrop(140),

    #Random horizontal flip
    transforms.RandomHorizontalFlip(p=0.5),

    # Turn the image into a torch.Tensor
    transforms.ToTensor(),

    # Normalize
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

BATCH_SIZE = 32
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size = BATCH_SIZE)

In [3]:
# Things to play with
# Hyperparameters -> NUM_EPOCHS - 100 | Learning_rate - 0.001 
# Functions -> Optimizer - Rprop | Loss Function - Cross Entropy Loss

NUM_EPOCHS = 100
lr = 0.001
device = get_device()

effnetb0 = EfficientNetB0()

loss_fn = nn.CrossEntropyLoss()

In [4]:
#Train EfficientNetV3 B0
effnetb0.to(device)
nadam_optim = torch.optim.NAdam(params=effnetb0.parameters(), lr=lr)
train_effnetb0_results, train_time_effnetb0 = train(effnetb0, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0096 | train_acc: 0.5195 | validation_loss: 1.0732 | validation_acc: 0.6449
Epoch: 2 | train_loss: 0.7030 | train_acc: 0.6723 | validation_loss: 0.6174 | validation_acc: 0.6965
Epoch: 3 | train_loss: 0.6273 | train_acc: 0.7133 | validation_loss: 0.6223 | validation_acc: 0.6309
Epoch: 4 | train_loss: 0.5761 | train_acc: 0.7282 | validation_loss: 0.9586 | validation_acc: 0.5664
Epoch: 5 | train_loss: 0.5246 | train_acc: 0.7550 | validation_loss: 0.5966 | validation_acc: 0.7254
Epoch: 6 | train_loss: 0.5141 | train_acc: 0.7588 | validation_loss: 1.4469 | validation_acc: 0.5684
Epoch: 7 | train_loss: 0.5427 | train_acc: 0.7635 | validation_loss: 0.5757 | validation_acc: 0.6805
Epoch: 8 | train_loss: 0.4790 | train_acc: 0.7857 | validation_loss: 0.4704 | validation_acc: 0.7715
Epoch: 9 | train_loss: 0.4695 | train_acc: 0.7810 | validation_loss: 1.4139 | validation_acc: 0.5742
Epoch: 10 | train_loss: 0.4562 | train_acc: 0.7953 | validation_loss: 0.4312 | validation_a

In [5]:
eval_effnetb0_results = eval_model(effnetb0, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))

  0%|          | 0/16 [00:02<?, ?it/s]

In [6]:
effnetb5 = EfficientNetB5()
#Train EfficientNetV3 B5
effnetb5.to(device)
nadam_optim = torch.optim.NAdam(params=effnetb5.parameters(), lr=lr)
train_effnetb5_results, train_time_effnetb5 = train(effnetb5, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3187 | train_acc: 0.3937 | validation_loss: 0.9301 | validation_acc: 0.5938
Epoch: 2 | train_loss: 0.9595 | train_acc: 0.5485 | validation_loss: 0.9436 | validation_acc: 0.5285
Epoch: 3 | train_loss: 0.8385 | train_acc: 0.6092 | validation_loss: 0.8765 | validation_acc: 0.5758
Epoch: 4 | train_loss: 0.8406 | train_acc: 0.6310 | validation_loss: 0.7315 | validation_acc: 0.6945
Epoch: 5 | train_loss: 0.8247 | train_acc: 0.6235 | validation_loss: 0.6734 | validation_acc: 0.6660
Epoch: 6 | train_loss: 0.7423 | train_acc: 0.6528 | validation_loss: 0.6408 | validation_acc: 0.6777
Epoch: 7 | train_loss: 0.7500 | train_acc: 0.6545 | validation_loss: 0.8845 | validation_acc: 0.6016
Epoch: 8 | train_loss: 0.7821 | train_acc: 0.6500 | validation_loss: 0.6139 | validation_acc: 0.7180
Epoch: 9 | train_loss: 0.6898 | train_acc: 0.6743 | validation_loss: 8.2936 | validation_acc: 0.3789
Epoch: 10 | train_loss: 0.6660 | train_acc: 0.6967 | validation_loss: 0.6022 | validation_a

In [10]:
model_name = Path("effnetb0_100epc.pth")
save_model(models_path,model_name,effnetb0)

model_name = Path("effnetb5_100epc.pth")
save_model(models_path,model_name,effnetb5)

Saving the model to: Models\effnetb0_100epc.pth
Saving the model to: Models\effnetb5_100epc.pth


In [4]:
model_name = Path("effnetb0_100epc.pth")
model_path = models_path/model_name
loaded_model_effnetb0 = EfficientNetB0()
loaded_model_effnetb0.load_state_dict(torch.load(model_path))
loaded_model_effnetb0.to(device)

EfficientNet(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (features): Sequential(
    (0): CNNBlock(
      (cnn): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (2): SiLU()
            (3): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (4): Sigmoid()
          )
        )
        (2): Conv2d(32, 16, kernel_size=(1, 1), stride=(

In [5]:
eval_effnetb0_results = eval_model(loaded_model_effnetb0, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))

  0%|          | 0/16 [00:01<?, ?it/s]

In [6]:
model_name = Path("effnetb5_100epc.pth")
model_path = models_path/model_name
loaded_model_effnetb5 = EfficientNetB5()
loaded_model_effnetb5.load_state_dict(torch.load(model_path))
loaded_model_effnetb5.to(device)

EfficientNet(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (features): Sequential(
    (0): CNNBlock(
      (cnn): Conv2d(3, 46, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): Conv2d(46, 46, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=46, bias=False)
          (bn): BatchNorm2d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): Conv2d(46, 11, kernel_size=(1, 1), stride=(1, 1))
            (2): SiLU()
            (3): Conv2d(11, 46, kernel_size=(1, 1), stride=(1, 1))
            (4): Sigmoid()
          )
        )
        (2): Conv2d(46, 24, kernel_size=(1, 1), stride

In [7]:
eval_effnetb5_results = eval_model(loaded_model_effnetb5, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 8.00 GiB total capacity; 7.07 GiB already allocated; 0 bytes free; 7.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
plot_loss_curves(train_effnetb0_results,"EfficientNet B0") 

NameError: name 'train_effnetb0_results' is not defined

In [ ]:
plot_loss_curves(train_effnetb5_results,"EfficientNet B5")

In [ ]:
plot_confusion_matrix(effnetb0,test_dl,device,class_names, "EfficientNet B0"), plot_confusion_matrix(effnetb5, test_dl, device, class_names, "EfficientNet B5")

In [9]:
all_results = pd.DataFrame([
    eval_effnetb0_results,
    ])

In [10]:
all_results

,model_name,model_loss,model_acc,model_inf_time
0,EfficientNet,7.698823,29.882812,66.912255
